In [ ]:
# build_graph_ui.py
# Run:  python build_graph_ui.py
# Output: inbox_graph.html

from pathlib import Path
from string import Template
import json

OUT = Path("inbox_graph.html")

# ----------------------------
# Replace with your real data
# ----------------------------
clusters = [
    {"id": "Product Launch",     "emails": 124, "participants": ["Alice", "Bob", "John"],      "spanDays": [0, 140]},
    {"id": "Marketing Strategy", "emails": 87,  "participants": ["Alice", "Karen"],            "spanDays": [10, 130]},
    {"id": "Hiring Discussions", "emails": 46,  "participants": ["Lisa", "John"],              "spanDays": [20, 135]},
    {"id": "Team Updates",       "emails": 61,  "participants": ["Lisa", "Karen", "Bob"],      "spanDays": [0, 140]},
    {"id": "Legal Notice",       "emails": 3,   "participants": ["Legal"],                     "spanDays": [100, 101]},
]

DEFAULTS = {
    "alpha_overview": 0.6,
    "tau": 7,
    "strong_w": 0.4,
    "densityPerc": {"minimal": 0.15, "normal": 0.35, "dense": 1.0}
}

template = Template(r"""<!doctype html>
<html>
<head>
  <meta charset="utf-8" />
  <title>Email Topic Graph</title>
  <style>
    html, body { height: 100%; margin: 0; }
    body {
      display: flex;
      font-family: system-ui, -apple-system, Segoe UI, Roboto, Arial, sans-serif;
    }
    #sidebar {
      width: 320px; padding: 16px; border-right: 1px solid #ddd;
      box-sizing: border-box; overflow-y: auto;
    }
    #network { flex: 1; }
    h3 { margin: 0 0 12px; }
    fieldset { border: 1px solid #ddd; margin-bottom: 12px; padding: 8px 12px; }
    fieldset > legend { padding: 0 6px; }
    .legend { font-size: 13px; margin-top: 16px; }
    .legend ul { padding-left: 18px; margin: 6px 0; }
    .preset-bar button {
      margin-right: 6px; margin-bottom: 6px; font-size: 12px; cursor: pointer;
    }
    label { font-size: 14px; }
  </style>
  <script src="https://unpkg.com/vis-network@9.1.2/dist/vis-network.min.js"></script>
  <link href="https://unpkg.com/vis-network@9.1.2/styles/vis-network.min.css" rel="stylesheet" />
</head>
<body>

<div id="sidebar">
  <h3>Email Topic Graph</h3>

  <div class="preset-bar">
    <button data-preset="balanced">Balanced overview</button>
    <button data-preset="people">People first</button>
    <button data-preset="time">Time first</button>
    <button data-preset="isolates">Triage isolates</button>
    <button data-preset="dense">Everything</button>
  </div>

  <fieldset>
    <legend>Primary view</legend>
    <label><input type="radio" name="view" value="overview" checked> Overview (Hybrid)</label><br>
    <label><input type="radio" name="view" value="people"> People Bridges</label><br>
    <label><input type="radio" name="view" value="time"> Time Windows</label><br>
    <label><input type="radio" name="view" value="isolates"> Isolates</label><br>
    <label><input type="radio" name="view" value="communities"> Communities</label>
  </fieldset>

  <fieldset>
    <legend>Detail level</legend>
    <label><input type="radio" name="detail" value="topics" checked> Topics only</label><br>
    <label><input type="radio" name="detail" value="topics-subtopics"> Topics + subtopics</label><br>
    <label><input type="radio" name="detail" value="drill"> Drill into emails on click</label>
  </fieldset>

  <fieldset>
    <legend>Density</legend>
    <label><input type="radio" name="density" value="minimal"> Minimal</label><br>
    <label><input type="radio" name="density" value="normal" checked> Normal</label><br>
    <label><input type="radio" name="density" value="dense"> Dense</label>
  </fieldset>

  <fieldset>
    <legend>Quick filters</legend>
    <label><input type="checkbox" id="hideWeak"> Hide weak edges</label><br>
    <label><input type="checkbox" id="hideIsolates"> Hide isolates</label>
  </fieldset>

  <fieldset>
    <legend>Color nodes by</legend>
    <label><input type="radio" name="colorBy" value="community" checked> Community</label><br>
    <label><input type="radio" name="colorBy" value="participant"> Top participant</label>
  </fieldset>

  <button id="apply">Apply</button>

  <div class="legend">
    <p><strong>Legend</strong></p>
    <ul>
      <li>Node = semantic topic cluster</li>
      <li>Edge = contextual relation (depends on chosen view)</li>
      <li>Strong edges: solid, thicker</li>
      <li>Weak edges: dashed, faint</li>
    </ul>
  </div>
</div>

<div id="network"></div>

<script>
const CLUSTERS = $CLUSTERS_JSON;
const DEFAULTS = $DEFAULTS_JSON;

function jaccard(a, b) {
  const A = new Set(a), B = new Set(b);
  if (A.size === 0 && B.size === 0) return 0;
  let inter = 0;
  for (const x of A) if (B.has(x)) inter++;
  return inter / (A.size + B.size - inter);
}

function timeDecay(deltaDays, tau) {
  return Math.exp(-deltaDays / tau);
}

function midpoint(span) {
  return (span[0] + span[1]) / 2.0;
}

function topPercentEdges(edges, perc) {
  if (perc >= 1.0) return edges;
  const sorted = [...edges].sort((a, b) => b.w - a.w);
  const keep = Math.max(1, Math.round(sorted.length * perc));
  return sorted.slice(0, keep);
}

function components(nodes, edges) {
  const id2idx = Object.fromEntries(nodes.map((n, i) => [n.id, i]));
  const adj = nodes.map(() => []);
  edges.forEach(e => {
    const u = id2idx[e.from], v = id2idx[e.to];
    adj[u].push(v); adj[v].push(u);
  });
  const comp = Array(nodes.length).fill(-1);
  let cid = 0;
  const nodeToComp = {};
  for (let i = 0; i < nodes.length; i++) {
    if (comp[i] !== -1) continue;
    const stack = [i];
    comp[i] = cid;
    while (stack.length) {
      const x = stack.pop();
      nodeToComp[nodes[x].id] = cid;
      for (const y of adj[x]) if (comp[y] === -1) {
        comp[y] = cid; stack.push(y);
      }
    }
    cid++;
  }
  return nodeToComp;
}

function topParticipant(participants) {
  return participants[0] || "none";
}

function build(view, densityKey, colorBy, hideWeak, hideIsolates) {
  const tau = DEFAULTS.tau;
  const strongW = DEFAULTS.strong_w;
  const perc = DEFAULTS.densityPerc[densityKey || "normal"] || 0.35;

  let edges = [];
  if (view !== "isolates") {
    for (let i = 0; i < CLUSTERS.length; i++) {
      for (let j = i + 1; j < CLUSTERS.length; j++) {
        const A = CLUSTERS[i], B = CLUSTERS[j];
        const people = jaccard(A.participants, B.participants);
        const ta = midpoint(A.spanDays), tb = midpoint(B.spanDays);
        const dt = Math.abs(tb - ta);
        const time = timeDecay(dt, tau);

        let w;
        if (view === "people") w = people;
        else if (view === "time") w = time;
        else w = DEFAULTS.alpha_overview * people + (1 - DEFAULTS.alpha_overview) * time;

        edges.push({ from: A.id, to: B.id, w, people, time, dt });
      }
    }
    edges = topPercentEdges(edges.filter(e => e.w > 0), perc);
  } else {
    edges = [];
  }

  if (hideWeak) edges = edges.filter(e => e.w >= strongW);

  const degree = Object.fromEntries(CLUSTERS.map(c => [c.id, 0]));
  edges.forEach(e => { degree[e.from] += e.w; degree[e.to] += e.w; });
  const maxDeg = Math.max(1, ...Object.values(degree));

  const palette = [
    "#1b6ee6","#ff7f0e","#2ca02c","#d62728","#9467bd",
    "#8c564b","#e377c2","#7f7f7f","#bcbd22","#17becf"
  ];
  const nodeToComp = components(CLUSTERS, edges);
  const partToColor = {};
  let partColorIdx = 0;

  const nodesVis = CLUSTERS
    .filter(n => !(hideIsolates && degree[n.id] === 0))
    .map(n => {
      const d = degree[n.id];
      const size = 15 + 35 * (d / maxDeg);
      let color;
      if (colorBy === "participant") {
        const tp = topParticipant(n.participants);
        if (!partToColor[tp]) partToColor[tp] = palette[(partColorIdx++) % palette.length];
        color = partToColor[tp];
      } else {
        const comp = nodeToComp[n.id] || 0;
        color = palette[comp % palette.length];
      }
      return {
        id: n.id,
        label: n.id,
        title: n.id + "<br/>" + n.emails + " mails",
        value: n.emails,
        size,
        color,
        font: { face: "Inter, Arial" }
      };
    });

  const visibleIds = new Set(nodesVis.map(n => n.id));
  const edgesVis = edges
    .filter(e => visibleIds.has(e.from) && visibleIds.has(e.to))
    .map(e => {
      const strong = e.w >= strongW;
      return {
        from: e.from, to: e.to,
        value: e.w,
        width: strong ? 4 + 6 * e.w : 1 + 2 * e.w,
        color: strong ? "rgba(27,110,230,1.0)" : "rgba(27,110,230,0.25)",
        dashes: !strong,
        smooth: { enabled: true, type: "dynamic", roundness: 0.2 },
        length: strong ? Math.max(120, parseInt(250 * (1 - e.w), 10)) : 380,
        title: "w=" + e.w.toFixed(2) +
               "<br/>people=" + e.people.toFixed(2) +
               "<br/>time=" + e.time.toFixed(2) +
               " (Δt=" + e.dt + "d)"
      };
    });

  return { nodesVis, edgesVis };
}

function readControls() {
  const view = document.querySelector('input[name="view"]:checked').value;
  const detail = document.querySelector('input[name="detail"]:checked').value;
  const density = document.querySelector('input[name="density"]:checked').value;
  const colorBy = document.querySelector('input[name="colorBy"]:checked').value;
  const hideWeak = document.getElementById('hideWeak').checked;
  const hideIsolates = document.getElementById('hideIsolates').checked;
  return { view, detail, density, colorBy, hideWeak, hideIsolates };
}

function applyPreset(name) {
  if (name === "balanced") {
    document.querySelector('input[name="view"][value="overview"]').checked = true;
    document.querySelector('input[name="density"][value="normal"]').checked = true;
    document.getElementById('hideWeak').checked = false;
    document.getElementById('hideIsolates').checked = false;
    document.querySelector('input[name="colorBy"][value="community"]').checked = true;
  } else if (name === "people") {
    document.querySelector('input[name="view"][value="people"]').checked = true;
    document.querySelector('input[name="density"][value="minimal"]').checked = true;
    document.getElementById('hideWeak').checked = true;
    document.getElementById('hideIsolates').checked = false;
    document.querySelector('input[name="colorBy"][value="community"]').checked = true;
  } else if (name === "time") {
    document.querySelector('input[name="view"][value="time"]').checked = true;
    document.querySelector('input[name="density"][value="normal"]').checked = true;
    document.getElementById('hideWeak').checked = false;
    document.getElementById('hideIsolates').checked = false;
    document.querySelector('input[name="colorBy"][value="community"]').checked = true;
  } else if (name === "isolates") {
    document.querySelector('input[name="view"][value="isolates"]').checked = true;
    document.querySelector('input[name="density"][value="dense"]').checked = true;
    document.getElementById('hideWeak').checked = false;
    document.getElementById('hideIsolates').checked = false;
    document.querySelector('input[name="colorBy"][value="community"]').checked = true;
  } else if (name === "dense") {
    document.querySelector('input[name="view"][value="overview"]').checked = true;
    document.querySelector('input[name="density"][value="dense"]').checked = true;
    document.getElementById('hideWeak').checked = false;
    document.getElementById('hideIsolates').checked = false;
    document.querySelector('input[name="colorBy"][value="community"]').checked = true;
  }
}

function render() {
  const { view, density, colorBy, hideWeak, hideIsolates } = readControls();
  const { nodesVis, edgesVis } = build(view, density, colorBy, hideWeak, hideIsolates);

  const container = document.getElementById('network');
  container.innerHTML = "";
  const data = { nodes: new vis.DataSet(nodesVis), edges: new vis.DataSet(edgesVis) };
  const options = {
    physics: {
      enabled: true,
      stabilization: { iterations: 200 },
      barnesHut: {
        gravitationalConstant: -3000,
        centralGravity: 0.2,
        springLength: 200,
        springConstant: 0.04,
        damping: 0.09,
        avoidOverlap: 0.1
      }
    },
    interaction: { hover: true, multiselect: true },
    edges: { smooth: { enabled: true, type: "dynamic", roundness: 0.2 } }
  };
  new vis.Network(container, data, options);
}

document.getElementById('apply').addEventListener('click', render);
document.querySelectorAll('.preset-bar button').forEach(btn => {
  btn.addEventListener('click', () => {
    applyPreset(btn.getAttribute('data-preset'));
    render();
  });
});

applyPreset('balanced');
render();
</script>

</body>
</html>
""")

html = template.substitute(
    CLUSTERS_JSON=json.dumps(clusters, separators=(",", ":")),
    DEFAULTS_JSON=json.dumps(DEFAULTS, separators=(",", ":")),
)

OUT.write_text(html, encoding="utf-8")
print(f"Wrote {OUT.resolve()}")
